# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Li  ->  J. Li  |  ['J. Li']
J. Shi  ->  J. Shi  |  ['J. Shi']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']


Arxiv has 85 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.19579
extracting tarball to tmp_2507.19579...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.19706
extracting tarball to tmp_2507.19706...

 done.


M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Retrieving document from  https://arxiv.org/e-print/2507.19727
extracting tarball to tmp_2507.19727... done.
Retrieving document from  https://arxiv.org/e-print/2507.19751


...

extracting tarball to tmp_2507.20744...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.20744/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/tab_outliers' from 'tmp_2507.20744/tables/tab_outliers.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/tab_sample' from 'tmp_2507.20744/tables/tab_sample.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/py

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.20744-b31b1b.svg)](https://arxiv.org/abs/2507.20744) | **Temperature based radial metallicity gradients in nearby galaxies**  |
|| <mark>K. Kreckel</mark>, et al. |
|*Appeared on*| *2025-07-29*|
|*Comments*| *12 pages, 6 figures, plus appendix. Accepted by A&A*|
|**Abstract**|            Gas-phase abundances provide insights into the baryon cycle, with radial gradients and 2D metallicity distributions tracking how metals build up and redistribute within galaxy disks over cosmic time. We use a catalog of 22,958 HII regions across 19 nearby spiral galaxies to examine how precisely the radial abundance gradients can be traced using only the [NII]5755 electron temperature as a proxy for `direct method' metallicities. Using 534 direct detections of the temperature sensitive [NII]5755 auroral line, we measure gradients in 15 of the galaxies. Leveraging our large catalog of individual HII regions, we stack in bins of HII region [NII]6583 luminosity and radius to recover stacked radial gradients. We find good agreement between the metallicity gradients from the stacked spectra, those gradients from individual regions and those from strong line methods. In addition, particularly in the stacked Te([NII]) measurements, some galaxies show very low (<0.05 dex) scatter in metallicities, indicative of a well-mixed ISM. We examine individual high confidence (S/N > 5) outliers and identify 13 regions across 9 galaxies with anomalously low metallicity, although this is not strongly reflected in the strong line method metallicities. By stacking arm and interarm regions, we find no systematic evidence for offsets in metallicity between these environments, suggesting enrichment within spiral arms is due to very localized processes. This work demonstrates the potential to systematically exploit the single [NII]5755 auroral line for detailed gas-phase abundance studies of galaxies. It provides strong validation of previous results, based on the strong line calibrations, of a well-mixed ISM across typical star-forming spiral galaxies.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19579-b31b1b.svg)](https://arxiv.org/abs/2507.19579) | **The phase spiral's origin and evolution: indications from its varying properties across the Milky Way disk**  |
|| A. Widmark, et al. |
|*Appeared on*| *2025-07-29*|
|*Comments*| *19 pages, 9 figures; appendix adds 7 pages, 8 figures*|
|**Abstract**|            The phase spiral is a perturbation to the vertical phase-space distribution of stars in the Milky Way disk. We study the phase spiral's properties and how they vary with spatial position, in order to constrain its origin and evolution, as well as properties of the disk itself. We produce high resolution maps using two complementary data processing schemes: (a) we bin the Gaia proper motion sample in a disk parallel spatial grid, reaching distances up to 4 kpc; (b) we bin the spatially nearby line-of-sight velocity sample in terms of disk parallel orbital parameters. We find complex structure, most significantly with respect to Galactocentric radius and guiding radius, but also in Galactic azimuth and epicyclic action and phase. We find that spiral winding and rotation phase vary smoothly across the disk, with close-to-flat radial profiles. This uniform structure, in particular for the rotation phase, indicates that the phase spiral was sourced by one or many global perturbations. Curiously, this also implies that the winding time has a strong slope with respect to Galactocentric radius, with low values for the inner disk.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19727-b31b1b.svg)](https://arxiv.org/abs/2507.19727) | **JWST Observations of SN 2023ixf II: The Panchromatic Evolution Between 250 and 720 Days After the Explosion**  |
|| K. Medler, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-07-29*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**|            We present the nebular phase spectroscopic and photometric observations of the nearby hydrogen-rich core-collapse supernova (CC-SN) 2023ixf, obtained through our JWST programs. These observations, combined with ground-based optical and near-infrared spectra, cover +252.67 - 719.96 d, creating a comprehensive, panchromatic time-series dataset spanning 0.32 - 30$\mu$m. In this second paper of the series, we focus on identifying key spectral emission features and tracking their evolution through the nebular phase. The JWST data reveal hydrogen emission from the Balmer to Humphreys series, as well as prominent forbidden lines from Ne, Ar, Fe, Co, and Ni. NIRSpec observations display strong emission from the first overtone and fundamental bands of carbon monoxide, which weaken with time as the ejecta cools and dust emission dominates. The spectral energy distribution shows a clear infrared excess emerging by +252.67 d peaking around 10.0$\mu$m, with a secondary bump at 18.0$\mu$m developing by +719.96 d. We suggest that this evolution could arises from multiple warm dust components. In upcoming papers in this series, we will present detailed modeling of the molecular and dust properties. Overall, this dataset significantly advances our understanding of the mid-infrared properties of CC-SNe, providing an unprecedented view of their late-time line, molecule, and dust emission.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19751-b31b1b.svg)](https://arxiv.org/abs/2507.19751) | **Phase II of the LAMOST-Kepler/K2 Survey. II. Time Domain of Medium-resolution Spectroscopic Observations from 2018 to 2023**  |
|| M. Qin, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-07-29*|
|*Comments*| *19 pages, 9 figures*|
|**Abstract**|            The LAMOST-Kepler/K2 Medium-Resolution Spectroscopic Survey (LK-MRS) conducted time-domain medium-resolution spectroscopic observations of 20 LAMOST plates in the Kepler and K2 fields from 2018 to 2023, a phase designated as LK-MRS-I. A catalog of stellar parameters for a total of 36,588 stars, derived from the spectra collected during these five years, including the effective temperature, the surface gravity, the metallicity, the {\alpha}-element abundance, the radial velocity, and v sin i of the target stars, is released, together with the weighted averages and uncertainties. At S/N = 10, the measurement uncertainties are 120 K, 0.18 dex, 0.13 dex, 0.08 dex, 1.9 km/s, and 4.0 km/s for the above parameters, respectively. Comparisons with the parameters provided by the APOGEE and GALAH surveys validate the effective temperature and surface gravity measurements, showing minor discrepancies in metallicity and {\alpha}-element abundance values. We identified some peculiar star candidates, including 764 metal-poor stars, 174 very metal-poor stars, and 30 high-velocity stars. Moreover, we found 2,333 stars whose radial velocity seems to be variable. Using Kepler/K2 or TESS photometric data, we confirmed 371 periodic variable stars among the radial velocity variable candidates and classified their variability types. LK-MRS-I provides spectroscopic data being useful for studies of the Kepler and K2 fields. The LK-MRS project will continue collecting time-domain medium-resolution spectra for target stars during the third phase of LAMOST surveys, providing data to support further scientific research.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.20206-b31b1b.svg)](https://arxiv.org/abs/2507.20206) | **New symbiotic stars or candidates in LAMOST low resolution spectra**  |
|| Y. Zhao, et al. -- incl., <mark>J. Li</mark>, <mark>J. Shi</mark> |
|*Appeared on*| *2025-07-29*|
|*Comments*| *18 pages, 12 figures, 5 tables*|
|**Abstract**|            Symbiotic stars are among the most crucial binary systems for studying binary star interactions and Type Ia supernova progenitors. Based on the unique observational characteristics of symbiotic stars, strong H I, He I emission lines, giant spectral features, and the presence of [O III], He II, O VI, and other emission lines with ionization potentials exceeding 35 eV, and the Gaia information, we search for new symbiotic stars using the low-resolution spectroscopic survey data from LAMOST. Thirty-six binary systems have been selected as symbiotic stars or candidates, in which the five known symbiotic stars are included. Among them five systems (ZTF J005917.52+315605.4, ATO J094137.5+075304, LAMOST J200310.90+360822.6, LAMOST J072528.18+342530.4, and V* V758 Cyg) have been found as new symbiotic stars. Notably, LAMOST J072528.18+342530.4 and V* V758 Cyg were also confirmed as new symbiotic stars in a recent study. For the remaining 26 candidates, classification is based solely on the presence of [O III] emission lines (with ionization potentials > 35 eV) and the absence of He II high-excitation emission lines. Further observations are needed to confirm their nature as symbiotic stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19706-b31b1b.svg)](https://arxiv.org/abs/2507.19706) | **MINERVA: A NIRCam Medium Band and MIRI Imaging Survey to Unlock the Hidden Gems of the Distant Universe**  |
|| A. Muzzin, et al. -- incl., <mark>M. Sharma</mark>, <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-07-29*|
|*Comments*| *29 pages, 12 Figures, submitted to ApJS. MINERVA Website: this https URL*|
|**Abstract**|            We present an overview of the MINERVA survey, a 259.8 hour (prime) and 127 hour (parallel) Cycle 4 treasury program on the James Webb Space Telescope (JWST). MINERVA is obtaining 8 filter NIRCam medium band imaging (F140M, F162M, F182M, F210M, F250M, F300M, F360M, F460M) and 2 filter MIRI imaging (F1280W, F1500W) in four of the five CANDELS Extragalactic fields: UDS, COSMOS, AEGIS and GOODS-N. These fields were previously observed in Cycle 1 with 7 - 9 NIRCam filters by the PRIMER, CEERS and JADES programs. MINERVA reaches a 5$\sigma$ depth of 28.1 mag in F300M and covers $\sim$ 542 arcmin$^2$, increasing the area of existing JWST medium-band coverage in at least 8 bands by $\sim$ 7$\times$. The MIRI imaging reaches a 5$\sigma$ depth of 23.9 mag in F1280W and covers $\sim$ 275 arcmin$^2$ in at least 2 MIRI filters. When combined with existing imaging, these data will provide a photometric catalog with 20-26 JWST filters (depending on field) and 26-35 filters total, including HST. This paper presents a detailed breakdown of the filter coverage, exposure times, and field layout relative to previous observations, as well as an overview of the primary science goals of the project. These include uncovering the physics of enigmatic sources hiding in current broadband catalogs, improving systematics on stellar mass functions and number densities by factors of $\gtrsim$ 3, and resolved mapping of stellar mass and star formation at 1 $< z <$ 6. When complete, MINERVA will become an integral part of the treasury deep field imaging datasets, significantly improving population studies with well-understood completeness, robust photometric redshifts, stellar masses, and sizes, and facilitating spectroscopic follow up for decades to come.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 44</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.20744/./figures/gradients_scal_v5.png', 'tmp_2507.20744/./figures/stacks_arm_interarm.png', 'tmp_2507.20744/./figures/compare_KK_RRV_MB_Chaos_v4.png']
copying  tmp_2507.20744/./figures/gradients_scal_v5.png to _build/html/
copying  tmp_2507.20744/./figures/stacks_arm_interarm.png to _build/html/
copying  tmp_2507.20744/./figures/compare_KK_RRV_MB_Chaos_v4.png to _build/html/
exported in  _build/html/2507.20744.md
    + _build/html/tmp_2507.20744/./figures/gradients_scal_v5.png
    + _build/html/tmp_2507.20744/./figures/stacks_arm_interarm.png
    + _build/html/tmp_2507.20744/./figures/compare_KK_RRV_MB_Chaos_v4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\niion}{[N \textsc{i}]}$
$\newcommand{\hei}{[He \textsc{i}]}$
$\newcommand{\siii}{[S \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\neii}{[Ne \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\niiauroral}{[N \textsc{ii}]\lambda5755}$
$\newcommand{\doh}{\Delta(O/H)}$
$\newcommand{\sigmaoh}{\sigma(O/H)}$
$\newcommand{\te}{T_{\rm e}}$
$\newcommand{\logten}{log_{10}}$
$\newcommand{\nee}{n_{\rm e}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegie} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\ESOChile}{\label{ESOChile} European Southern Observatory, Avenida Alonso de Cordoba 3107, Casilla 19, Santiago 19001, Chile}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\UCatolica}{\label{UCatolica} Instituto de Astronomía, Universidad Católica del Norte, Av. Angamos 0610, Antofagasta, Chile}$
$\newcommand{\UT}{\label{UT} McDonald Observatory, The University of Texas at Austin, 1 University Station, Austin, TX 78712-0259, USA}$
$\newcommand{\Vanderbilt}{\label{Vanderbilt} Department of Physics and Astronomy, Vanderbilt University, VU Station 1807, Nashville, TN 37235, USA}$
$\newcommand{\UNF}{\label{UNF} Department of Physics, University of North Florida, 1 UNF Dr. Jacksonville FL 32224}$
$\newcommand{\NAOC}{\label{NAOC} Chinese Academy of Sciences South America Center for Astronomy, National Astronomical Observatories, CAS, Beijing 100101, China}$
$\newcommand{\CASA}{\label{CASA} Center for Astrophysics and Space Astronomy, University of Colorado, 389 UCB, Boulder, CO 80309-0389, USA}$
$\newcommand{\UNAM}{\label{UNAM} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 106, Ensenada 22800, BC, México}$
$\newcommand{\UDP}{\label{UDP} Instituto de Estudios Astrofísicos, Facultad de Ingeniería y Ciencias, Universidad Diego Portales, Av. Ejército Libertador 441, Santiago, Chile}$
$\newcommand{\Steward}{\label{Steward} Steward Observatory, University of Arizona, 933 N. Cherry Ave., Tucson, AZ 85721-0065, USA}$
$\newcommand{\APO}{\label{APO} Apache Point Observatory and New Mexico State University, P.O. Box 59,$
$Sunspot, NM 88349-0059, USA}$
$\newcommand{\UNAMCU}{\label{UNAMCU} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 70-264, CDMX 04510, México}$
$\newcommand{\UWash}{\label{UWash} Department of Astronomy, University of Washington, Seattle, WA, 98195}$
$\newcommand{Ç}{\label{CC} Department of Physics, Colorado College, Colorado Springs, CO 80903}$
$\newcommand{\Utah}{\label{Utah} Department of Physics and Astronomy, University of Utah, 115 S. 1400 E., Salt Lake City, UT 84112, USA}$
$\newcommand{\UConcepcion}{\label{UConcepcion} Departamento de Astronomía, Universidad de Concepción, Casilla 160-C, Concepción, Chile}$
$\newcommand{\FCLA}{\label{FCLA} Franco-Chilean Laboratory for Astronomy, IRL 3386, CNRS and Universidad de Chile, Santiago, Chile}$
$\newcommand{\Oklahoma}{\label{Oklahoma} Homer L. Dodge Department of Physics and Astronomy, University of Oklahoma, Norman, OK 73019, USA}$
$\newcommand{\UIUC}{\label{UIUC} Department of Astronomy, University of Illinois, Urbana, IL 61801, USA}$
$\newcommand{\Harvard}{\label{Harvard} Harvard-Smithsonian Center for Astrophysics, Cambridge, MA 02138, USA}$
$\newcommand{\caltech}{\label{caltech} Department of Astronomy, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{\UOA}{\label{UOA} Department of Physics, University of Arkansas, 226 Physics Building, 825 West Dickson Street, Fayetteville, AR 72701, USA}$
$\newcommand{\trieste}{\label{trieste} Department of Physics, Astronomy Section, University of Trieste, Via G.B. Tiepolo, 11, I-34143 Trieste, Italy}$
$\newcommand{\Rad}{\label{Rad}{ Elizabeth S. and Richard M. Cashin Fellow at the Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\UCT}{\label{UCT}{ Department of Astronomy, University of Cape Town, Rondebosch 7701, Cape Town, South Africa}}$
$\newcommand{\Oxford}{\label{Oxford}{ Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\natexlab{#1}$</div>



<div id="title">

#    Temperature based radial metallicity gradients in nearby galaxies   

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.20744-b31b1b.svg)](https://arxiv.org/abs/2507.20744)<mark>Appeared on: 2025-07-29</mark> -  _12 pages, 6 figures, plus appendix. Accepted by A&A_

</div>
<div id="authors">

<mark>K. Kreckel</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Gas-phase abundances provide insights into the baryon cycle, with radial gradients and 2D metallicity distributions tracking how metals build up and redistribute within galaxy disks over cosmic time. We use a catalog of 22,958 $\hii$ regions across 19 nearby spiral galaxies to examine how precisely the radial abundance gradients can be traced using only the $\niiauroral$ electron temperature as a proxy for `direct method' metallicities. Using 534 direct detections of the temperature sensitive $\niiauroral$ auroral line, we measure gradients in 15 of the galaxies. Leveraging our large catalog of individual HII regions, we stack in bins of $\hii$ region $\nii$ $\lambda$ 6583 luminosity and radius to recover stacked radial gradients. We find good agreement between the metallicity gradients from the stacked spectra,  those gradients from individual regions and those from strong line methods. In addition, particularly in the stacked $\te$ $\nii$ measurements, some galaxies show very low ( $<$ 0.05 dex) scatter in metallicities, indicative of a well-mixed ISM. We examine individual high confidence (S/N $>$ 5) outliers and identify 13 regions across 9 galaxies with anomalously low metallicity, although this is not strongly reflected in the strong line method metallicities. By stacking arm and interarm regions, we find no systematic evidence for offsets in metallicity between these environments, suggesting enrichment within spiral arms is due to very localized processes. This work demonstrates the potential to systematically exploit the single $\niiauroral$ auroral line for detailed gas-phase abundance studies of galaxies. It provides strong validation of previous  results, based on the strong line calibrations, of a well-mixed ISM across typical star-forming spiral galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2507.20744/./figures/gradients_scal_v5.png" alt="Fig5" width="100%"/>

**Figure 5. -** A comparison of metallicity gradients. The Scal values (grey) are compared with individual $\te$$\nii$ metallicities (red) and $\te$$\nii$ stacked metallicities (black). For context, the subsample of individual regions with $\te$$\nii$ detections are highlighted within the Scal measurements in light blue. Linear radial gradients are fit when there are at least 5 measurements that cover at least 0.5 r$_{\rm eff}$. Note that $\hii$ regions in NGC 4254 and NGC 4535 do not cover the full disk due to the AO notch filter. Galaxies are ordered from low (top-left) to high (bottom-right) stellar mass, and axis scalings are not  matched  between galaxies.   (*fig:gradients_scal*)

</div>
<div id="div_fig2">

<img src="tmp_2507.20744/./figures/stacks_arm_interarm.png" alt="Fig2" width="100%"/>

**Figure 2. -** A comparison of $\te$$\nii$ measured in stacked arm and interarm environments. We color-code the points by L($\nii$) (left) and radius (right) but see no evidence for trends or bulk offsets between environments.  (*fig:stack-arm-interarm*)

</div>
<div id="div_fig3">

<img src="tmp_2507.20744/./figures/compare_KK_RRV_MB_Chaos_v4.png" alt="Fig3" width="100%"/>

**Figure 3. -** A comparison of $\te$$\nii$ measurements used in this work \citep{Brazzini2024} with three different sets of measurements available in the literature. While auroral line fits in \cite{Kreckel2022}(left) and \cite{RRV2024}(center) are both based on the same underlying MUSE data set, different assumptions have been made in the SSP fitting and region boundaries. \cite{Berg2015} measurements (right) are based on independent long-slit spectroscopy. As there are only five regions in common with \cite{Berg2015}, no offset or scatter is calculated in comparison with this sample. (*fig:compare_Te*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.20744"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

151  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
